In [1]:
# !pip install transformers
# !pip install git+https://github.com/yuchenlin/LLM-Blender.git
# !pip install "pydantic<2"
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
from torch import nn
from transformers import Trainer
import torch.nn.functional as F
import torchvision.transforms as tr # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리
import csv

from llm_blender.pair_ranker.pairrm import DebertaV2PairRM # or copy the DebertaV2PairRM definition here, https://github.com/yuchenlin/LLM-Blender/blob/main/llm_blender/pair_ranker/pairrm.py
from transformers import AutoTokenizer
from typing import List

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

2024-02-12 07:52:23.407429: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-12 07:52:24.513546: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-02-12 07:52:24.513663: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-02-12 07:52:24.513677: W tensorfl


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


Using device: cuda


In [3]:
test_df = pd.read_csv("./data/test.csv")
# test_df

In [4]:
# count = test_df['질문'].str.count('또한')
# count[count>0]

In [5]:

appedix_dict_1 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('?') >=2:
                appendix = row[q_col].split('?')[1] + "?"
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('?')[0] + "?"
                row[q_col] = row[q_col].strip()
                appedix_dict_1[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()
appedix_dict_1   

{0: '또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?',
 11: '또한, 부실 시공으로 인해 타공하자가 발생할 가능성이 있나요?',
 16: '그리고 유성페인트를 사용하는 것에 대한 부작용이 있을까요?',
 17: '그리고 반점이 생긴지 1년 이내인 하자에 대해 어떤 보수작업을 해야 하나요?',
 20: '또한, 피스 하자가 무엇인지 자세히 설명해 주실 수 있나요?',
 22: '또한, 겨울에 도배를 할 때 특별히 신경써야 할 사항이 있을까요?',
 23: '또한, 페인트가 남으면 어디에 보관하는 게 좋을까요?',
 24: '또한, 유광 자기질 타일은 얼마나 오랫동안 사용할 수 있을까요?',
 25: '그리고 스탠드조명을 선택할 때 주의할 점이 있을까요?',
 26: '그리고 불량 도배지를 사용할 경우 도배지가 얼마나 오랫동안 버틸까요?',
 27: '그리고 실크벽지의 교체 주기는 얼마나인가요?',
 28: '그리고 철골구조의 장점은 무엇인가요?',
 29: '그리고 조적식 구조란 무엇인가요?',
 33: '또한, 액체방수공사는 무엇을 하는 것인가요?',
 35: '그리고 장판이 남을 때 어떻게 처리해야 하나요?',
 38: '또한, 콘크리트 벽에 구멍을 뚫는 방법에는 어떤 도구나 기술을 사용해야 하나요?',
 39: '그리고 인테리어에 가장 많이 사용되는 도배재료는 무엇인가요?',
 42: '그리고 복도나 협소한 공간을 확장시키기 위해 가장 효과적인 방법이 무엇일까요?',
 48: '또한, 배관공사 시 통기구를 설치해야 하는 이유가 무엇인가요?',
 49: '또한, 규산질계 침투성 도포 방수공사는 어떤 방식으로 이루어지나요?',
 50: '또한, 내부와 외부 온도의 큰 차이로 인해 곰팡이 발생이 빨라지나요?',
 54: '그리고 기둥-보 구조 방식은 무엇을 의미하나요?',
 56: '그리고 새집증후군을 예방하는 데 가장 효과적인 방법은 무엇인가요?',
 59: '또한, 옥상 방수용 탄성 에멀전 페인트를 사용하는 장점은 무엇인가요?',
 63: 

In [6]:

appedix_dict_2 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('그리고') > 0 and row[q_col].count('이를') == 0 :
                appendix = row[q_col].split('그리고')[1]
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('그리고')[0]
                row[q_col] = row[q_col].strip()
                appedix_dict_2[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()
appedix_dict_2     

{6: '합지벽지의 어떤 단점이 있나요?',
 15: '페인트 상도재의 역할과 사용 방법에 대해 알려주세요.',
 47: '습도가 높을 때 곰팡이가 어떻게 발생하는지 자세히 알고 싶습니다.',
 55: '도배지가 남으면 어떻게 처리해야 하나요?',
 64: '소화기 종류에는 어떤 것들이 있는지 알려주세요.',
 86: '토목이 무엇인지 설명해줘.',
 93: '셀룰로오스의 단점에 대해 설명해주세요.',
 94: '반려동물을 위한 바닥재에는 어떤 종류가 있는지 알려주세요.',
 98: '밀풀 사용 시 주의할 점은 무엇인가요?',
 99: '통나무구조 방식은 어떤 건물에 사용되는 건축 구조 방식인가요?',
 108: '펫테리어가 무엇인지 자세히 알려주세요.',
 119: '철근콘크리트 구조에 대해 좀 더 자세히 알려주세요.'}

In [7]:
appedix_dict_3 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('또한') > 0:
                appendix = row[q_col].split('또한')[1] + "또한"
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('또한')[0] + "또한"
                row[q_col] = row[q_col].strip()
                appedix_dict_3[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()

appedix_dict_3

{2: ', 압출법 단열판을 사용하는 것의 장점은 무엇인가요?또한',
 32: ', 세라믹 타일을 사용할 때 고려해야 할 단점은 무엇인가요?또한',
 45: ', 낡은 목재 가구의 흠집을 숨기는 방법을 알려주세요.또한',
 52: ', 속건형 유성 발수제의 사용 목적과 효과에 대해 알려주세요.또한',
 72: ', 강화마루의 장점은 무엇입니까?또한',
 89: ', 아이소핑크의 장점은 무엇인가요?또한'}

In [8]:
test_df['질문']

0                       방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?
1                도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2                                     큐블럭의 단점을 알려주세요. 또한
3      철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있...
4                       도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
                             ...                        
125                      분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요?
126                            압출법 보온판의 가장 큰 장점은 무엇인가요?"
127    평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128    석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...
129                                 카페트의 기대 수명은 얼마나 될까요?
Name: 질문, Length: 130, dtype: object

In [9]:

                  
tokenizer = PreTrainedTokenizerFast.from_pretrained('./skt_augment_3_epoch/', eos_token='</s>')
model = GPT2LMHeadModel.from_pretrained("./skt_augment_3_epoch/")
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [10]:

# Inference를 위한 test.csv 파일 로드
# test = pd.read_csv('./data/test.csv')

# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
file_path = "result_skt_r2.txt"
total_preds = []
# '질문' 컬럼의 각 질문에 대해 답변 생성
# for idx,model in enumerate(model_list):
preds = []
idx = 0
for test_question in tqdm(test_df['질문']):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=2000,
            min_length=20,
            temperature=0.8,
            top_k=150,
            top_p=0.8,
            repetition_penalty=1.0,
            do_sample=True,
            num_beams=150,
            # no_repeat_ngram_size=3,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        print(answer_only)
        preds.append(answer_only)
        answer_only = answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1


    # with open(csv_file_path, mode='w') as csv_file:
    #     csv_writer = csv.writer(csv_file)
    #     csv_writer.writerow(answer_only)
# break
# total_preds.append(preds)
# model = model.cpu()
# torch.cuda.empty_cache()

# 원목마루와 롱브릭타일에 대해 설명해주세요. 각각의 단점과 특징은 무엇인가요?


  1%|█▎                                                                                                                                                                         | 1/130 [00:13<30:03, 13.98s/it]

방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?</s> 방청 페인트는 다양한 종류로 분류됩니다. 그 종류에는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트가 각자의 특성과 용도에 맞게 사용됩니다.</s>
방청 페인트는 다양한 종류로 분류됩니다. 그 종류에는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트가 각자의 특성과 용도에 맞게 사용됩니다.


  2%|██▌                                                                                                                                                                      | 2/130 [01:00<1:10:41, 33.14s/it]

도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?</s> 도배지에 녹이 발생하는 원인은 공간 내 습도가 높거나 누수가 발생하여 발생할 수 있습니다. 이로 인해 도배지 안쪽의 금속이 녹아서 녹이 도배지에 베어나오게 됩니다. 습기가 높은 상태가 지속될 경우 녹오염이 발생할 수 있으며, 또한 누수에 의해 도배지가 젖은 상태가 유지될 경우 곰팡이가 발생할 수 있습니다. 이러한 경우에는 먼저, 제습기를 가동하거나 환기를 통해 공간 내 적정한 습도를 유지하는 것이 중요합니다. 또한, 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체하는 것이 필요할 수 있습니다. 이러한 보수작업은 전문가의 도움을 받아 진행하는 것이 좋습니다.</s>
도배지에 녹이 발생하는 원인은 공간 내 습도가 높거나 누수가 발생하여 발생할 수 있습니다. 이로 인해 도배지 안쪽의 금속이 녹아서 녹이 도배지에 베어나오게 됩니다. 습기가 높은 상태가 지속될 경우 녹오염이 발생할 수 있으며, 또한 누수에 의해 도배지가 젖은 상태가 유지될 경우 곰팡이가 발생할 수 있습니다. 이러한 경우에는 먼저, 제습기를 가동하거나 환기를 통해 공간 내 적정한 습도를 유지하는 것이 중요합니다. 또한, 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체하는 것이 필요할 수 있습니다. 이러한 보수작업은 전문가의 도움을 받아 진행하는 것이 좋습니다.


  2%|███▉                                                                                                                                                                       | 3/130 [01:08<45:22, 21.44s/it]

...

평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어떤 장점이 있나요?</s> 평지붕은 옥상을 활용할 수 있다는 점이 장점이며, 쾌적한 휴식 공간, 정원, 혹은 태양광 발전 설비를 만들 수 있는 공간으로 활용할 수 있습니다. 또한 건축비용이 다른 형태의 지붕에 비해 상대적으로 낮고, 유지보수가 용이하며 안전한 작업공간으로 활용 가능합니다.</s>
평지붕은 옥상을 활용할 수 있다는 점이 장점이며, 쾌적한 휴식 공간, 정원, 혹은 태양광 발전 설비를 만들 수 있는 공간으로 활용할 수 있습니다. 또한 건축비용이 다른 형태의 지붕에 비해 상대적으로 낮고, 유지보수가 용이하며 안전한 작업공간으로 활용 가능합니다.


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 129/130 [30:39<00:17, 17.14s/it]

석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 알려주세요."</s> 석고 수정은 벽을 이루고 있는 석고에 훼손이 생겼을 때 발생하는 현상을 말합니다. 이는 건물이 오래되거나 지진 등 물리적인 변화, 습기의 영향, 그리고 건물 자체의 구조적 결함으로 인해 발생할 수 있습니다. 석고 수정이 발생하는 경우, 해당 부분을 수정하고 전문가에게 정밀 진단을 받아야 합니다. 또한, 습도를 관리하고 제습기를 가동하여 실내 적정 습도를 유지하는 것이 중요합니다. 구조적 결함에 의한 석고 수정이 발생한 경우에는 건물의 구조적 결함을 우선적으로 해결하고 석고 수정을 보수하는 것이 중요합니다.</s>
석고 수정은 벽을 이루고 있는 석고에 훼손이 생겼을 때 발생하는 현상을 말합니다. 이는 건물이 오래되거나 지진 등 물리적인 변화, 습기의 영향, 그리고 건물 자체의 구조적 결함으로 인해 발생할 수 있습니다. 석고 수정이 발생하는 경우, 해당 부분을 수정하고 전문가에게 정밀 진단을 받아야 합니다. 또한, 습도를 관리하고 제습기를 가동하여 실내 적정 습도를 유지하는 것이 중요합니다. 구조적 결함에 의한 석고 수정이 발생한 경우에는 건물의 구조적 결함을 우선적으로 해결하고 석고 수정을 보수하는 것이 중요합니다.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 130/130 [30:45<00:00, 14.20s/it]

카페트의 기대 수명은 얼마나 될까요?</s> 카페트의 기대수명은 주로 6년이며, 사용빈도, 관리정도, 실내환경 등에 따라 달라질 수 있습니다. 일반적으로 주기적인 청소와 정기적인 유지보수를 통해 카페트의 수명을 연장시킬 수 있습니다.</s>
카페트의 기대수명은 주로 6년이며, 사용빈도, 관리정도, 실내환경 등에 따라 달라질 수 있습니다. 일반적으로 주기적인 청소와 정기적인 유지보수를 통해 카페트의 수명을 연장시킬 수 있습니다.


In [11]:
file_path = "result_skt_appendix_1_r2.txt"


for idx, test_question in tqdm(appedix_dict_1.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=2000,
            min_length=20,
            temperature=0.8,
            top_k=150,
            top_p=0.8,
            repetition_penalty=1.0,
            do_sample=True,
            num_beams=150,
            # no_repeat_ngram_size=3,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  2%|███▋                                                                                                                                                                        | 1/46 [00:04<03:27,  4.60s/it]

또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?</s> 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.</s>


  4%|███████▍                                                                                                                                                                    | 2/46 [00:14<05:37,  7.66s/it]

또한, 부실 시공으로 인해 타공하자가 발생할 가능성이 있나요?</s> 적절하지 않은 도구로 작업하거나 타공부위의 크기를 정확히 측정하지 않고 작업하면 타공불량이 발생할 수 있습니다.</s>


  7%|███████████▏                                                                                                                                                                | 3/46 [00:23<05:48,  8.11s/it]

...

그리고 내진구조란 무엇인가요?</s> 내진구조는 지진으로 인한 건축물의 피해를 최소화하기 위해 만들어진 건축물의 아름다움을 말합니다. 내진구조를 갖춘 건물은 지진 발생 시 건물의 더 큰 피해를 피할 수 있도록 설계되어 있습니다. 내진벽과 같은 부재를 설치하여 건물을 보다 튼튼하게 만들어 지진으로 인한 피해를 최소화하는 것이 내진구조의 주요 목적입니다.</s>


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎   | 45/46 [08:17<00:09,  9.30s/it]

그리고 아파트 도배 평수를 어떻게 계산해야 하나요?</s> 일반적으로 도배 평수는 주택 평수에 3. 5를 곱하여 계산합니다. 따라서, 주택 평수가 30평이라면 도배평수는 30평 * = 이 됩니다. 이는 도배할 벽 및 천장의 면적과 관련된 비용을 계산하는 데 도움을 줄 수 있습니다.</s>


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46/46 [08:23<00:00, 10.95s/it]

그리고 오리지널징크의 장점에는 무엇이 있나요?</s> 오리지널징크는 일반 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 쉽다는 장점이 있습니다. 또한 오리지널징크는 가벼우면서도 매우 견고하며 낮은 유지보수 비용과 높은 재활용 가능성을 갖고 있습니다.</s>


In [12]:
file_path = "result_skt_appendix_2_r2.txt"


for idx, test_question in tqdm(appedix_dict_2.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=2000,
            min_length=20,
            temperature=0.8,
            top_k=150,
            top_p=0.8,
            repetition_penalty=1.0,
            do_sample=True,
            num_beams=150,
            # no_repeat_ngram_size=3,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  8%|██████████████▎                                                                                                                                                             | 1/12 [00:04<00:49,  4.46s/it]

합지벽지의 어떤 단점이 있나요?</s> 합지벽지의 경우 상대적으로 수명이 짧고 내구성이 약하여 금방 무너질 수 있으며, 오염이 발생하면 청소가 어렵고 변색이 생길 수 있다는 점입니다. 또한, 시간이 지남에 따라 빨리 훼손될 수도 있습니다.</s>


 17%|████████████████████████████▋                                                                                                                                               | 2/12 [00:12<01:03,  6.34s/it]

페인트 상도재의 역할과 사용 방법에 대해 알려주세요.</s> 페인트 상도재는 최종 마감으로 칠하는 페인트로, 영어로는 Top Coating이라고 합니다. 상도재는 하도재와 주도재의 제품을 보호하는 역할을 담당하며, 외부 표면에 노출될 때의 열 및 기후에 강하며 내구성이 높은 특징을 가지고 있습니다. 이러한 장점 때문에 가격이 다소 비쌉니다.</s>


 25%|███████████████████████████████████████████                                                                                                                                 | 3/12 [00:20<01:05,  7.26s/it]

...

통나무구조 방식은 어떤 건물에 사용되는 건축 구조 방식인가요?</s> 통나무구조 방식은 주로 원형이나 각형의 내력벽으로 구성되며, 목재 바닥이나 약 2x4 경량 목구조와 유사한 방식으로, 이 구조는 주로 통나무를 쌓는 형태이기 때문에 1층 높이에서 7 ~ 12cm 정도의 침하가 장기적으로 발생할 수 있으므로, 창문틀 등의 개구부에는 침하를 고려한 디테일을 고려해야 합니다. 통나무구조 방식은 자연적이고 활기찬 분위기를 연출해내기 위해 주택 및 휴양시설에서 선호되는 구조 방식 중 하나입니다.</s>


 92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊              | 11/12 [01:20<00:09,  9.68s/it]

펫테리어가 무엇인지 자세히 알려주세요.</s> 펫테리어란 ‘반려동물(pet)’과 ‘인테리어(interior)’의 합성어로, 반려동물과의 생활을 고려한 편의성을 높이면서도 주거 공간의 인테리어를 고려하는 스타일을 말합니다. 즉, 반려동물과 관련된 편의시설과 인테리어 소품을 활용하여 쾌적하고 안락한 공간을 조성하는 것이 펫테리어의 핵심입니다. 이를 통해 주거 환경을 반려동물과 함께하는 가족들에게 쾌적한 공간을 제공하는 것이 목표입니다.</s>


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [01:28<00:00,  7.42s/it]

철근콘크리트 구조에 대해 좀 더 자세히 알려주세요.</s> 철근콘크리트 구조는 콘크리트의 압축 강도를 살려내는 동시에 철근으로 인장 강도를 보완하여 구축된 구조체입니다. 일반적으로 도심에서 많이 사용되며 내구성, 내화 및 내진 성능이 우수합니다. 그러나 고층 건물에서는 아래층의 보나 기둥이 상당한 무게를 지탱해야 하므로 구조 요소의 단면이 커야 하는 문제가 있습니다.</s>


In [13]:
file_path = "result_skt_appendix_2_r2.txt"


for idx, test_question in tqdm(appedix_dict_2.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=2000,
            min_length=20,
            temperature=0.8,
            top_k=150,
            top_p=0.8,
            repetition_penalty=1.0,
            do_sample=True,
            num_beams=150,
            # no_repeat_ngram_size=3,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  8%|██████████████▎                                                                                                                                                             | 1/12 [00:04<00:54,  4.96s/it]

합지벽지의 어떤 단점이 있나요?</s> 합지벽지의 경우 상대적으로 수명이 짧고 내구성이 약하여 금방 무너질 수 있으며, 오염이 발생하면 청소가 어렵고 변색이 생길 수 있다는 점입니다. 또한, 시간이 지남에 따라 빨리 훼손될 수도 있습니다.</s>


 17%|████████████████████████████▋                                                                                                                                               | 2/12 [00:12<01:05,  6.51s/it]

페인트 상도재의 역할과 사용 방법에 대해 알려주세요.</s> 페인트 상도재는 최종 마감으로 칠하는 페인트로, 영어로는 Top Coating이라고 합니다. 상도재는 하도재와 주도재의 제품을 보호하는 역할을 담당하며, 외부 표면에 노출될 때의 열 및 기후에 강하며 내구성이 높은 특징을 가지고 있습니다. 이러한 장점 때문에 가격이 다소 비쌉니다.</s>


 25%|███████████████████████████████████████████                                                                                                                                 | 3/12 [00:20<01:03,  7.07s/it]

...

통나무구조 방식은 어떤 건물에 사용되는 건축 구조 방식인가요?</s> 통나무구조 방식은 주로 원형이나 각형의 내력벽으로 구성되며, 목재 바닥이나 약 2x4 경량 목구조와 유사한 방식으로, 이 구조는 주로 통나무를 쌓는 형태이기 때문에 1층 높이에서 7 ~ 12cm 정도의 침하가 장기적으로 발생할 수 있으므로, 창문틀 등의 개구부에는 침하를 고려한 디테일을 고려해야 합니다. 통나무구조 방식은 자연적이고 활기찬 분위기를 연출해내기 위해 주택 및 휴양시설에서 선호되는 구조 방식 중 하나입니다.</s>


 92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊              | 11/12 [01:23<00:10, 10.30s/it]

펫테리어가 무엇인지 자세히 알려주세요.</s> 펫테리어란 ‘반려동물(pet)’과 ‘인테리어(interior)’의 합성어로, 반려동물과의 생활을 고려한 편의성을 높이면서도 주거 공간의 인테리어를 고려하는 스타일을 말합니다. 즉, 반려동물과 관련된 편의시설과 인테리어 소품을 활용하여 쾌적하고 안락한 공간을 조성하는 것이 펫테리어의 핵심입니다. 이를 통해 주거 환경을 반려동물과 함께하는 가족들에게 쾌적한 공간을 제공하는 것이 목표입니다.</s>


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [01:30<00:00,  7.58s/it]

철근콘크리트 구조에 대해 좀 더 자세히 알려주세요.</s> 철근콘크리트 구조는 콘크리트의 압축 강도를 살려내는 동시에 철근으로 인장 강도를 보완하여 구축된 구조체입니다. 일반적으로 도심에서 많이 사용되며 내구성, 내화 및 내진 성능이 우수합니다. 그러나 고층 건물에서는 아래층의 보나 기둥이 상당한 무게를 지탱해야 하므로 구조 요소의 단면이 커야 하는 문제가 있습니다.</s>


In [14]:
file_path = "result_skt_appendix_3_r2.txt"


for idx, test_question in tqdm(appedix_dict_3.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=2000,
            min_length=20,
            temperature=0.8,
            top_k=150,
            top_p=0.8,
            repetition_penalty=1.0,
            do_sample=True,
            num_beams=150,
            # no_repeat_ngram_size=3,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )
        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.2,
        #     top_k=150,
        #     top_p=1.2,
        #     repetition_penalty=1.4,
        #     do_sample=True,
        #     num_beams=300,
        #     no_repeat_ngram_size=6,
        #     # num_return_sequences=1,
        #     eos_token_id=tokenizer.eos_token_id
        # )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+5:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

 17%|████████████████████████████▊                                                                                                                                                | 1/6 [00:07<00:39,  7.86s/it]

, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?또한</s> 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.</s>


 33%|█████████████████████████████████████████████████████████▋                                                                                                                   | 2/6 [00:11<00:20,  5.23s/it]

, 세라믹 타일을 사용할 때 고려해야 할 단점은 무엇인가요?또한</s> 세라믹은 저온으로 구워내기에 온도 변화에 민감하여 내장용으로만 사용하고 경도가 약해 충격과 마모에 약하다는 단점이 있습니다.</s>


 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 3/6 [00:22<00:24,  8.07s/it]

...

, 속건형 유성 발수제의 사용 목적과 효과에 대해 알려주세요.또한</s> 속건형 유성의 장점은 건축자재의 표면에 도장되어 빗물 및 다른 액체의 침투를 막아주어 건축물을 보호하는 것뿐만 아니라, 건축자재의 색상을 오랫동안 유지시켜주고, 균열 현상과 백화 현상을 방지하여 건축물의 수명을 연장시킨다는 점입니다. 또한, 이것으로 철근의 부식도 방지할 수 있어 건물의 구조적 안전성을 향상시킵니다.</s>


 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                            | 5/6 [00:40<00:08,  8.35s/it]

, 강화마루의 장점은 무엇입니까?또한</s> 오리지널징크의 장점은 다양한 패턴과 디자인을 적용할 수 있다는 점입니다. 또한 친환경적이고 높은 내식성이 있어 수명이 다른 징크에 비해 길며 유지보수가 쉽다는 점이 있습니다.</s>


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:47<00:00,  7.86s/it]

, 아이소핑크의 장점은 무엇인가요?또한</s> 아이소핑크의 경우 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중에서 열전도율이 가장 큰 이유로 시공이 어려운 점입니다. 또한, 경량으로 제작되어 운반 및 설치가 용이하며 내화성과 내열성이 뛰어나 안전성이 높다는 특징을 가지고 있습니다.</s>


In [15]:


# source_prefix = "<|source|>"
# cand1_prefix = "<|candidate1|>"
# cand2_prefix = "<|candidate2|>"
# inputs = ["hello!", "I love you!"]
# candidates_A = ["hi!", "I hate you!"]
# candidates_B = ["f**k off!", "I love you, too!"]
# def tokenize_pair(sources:List[str], candidate1s:List[str], candidate2s:List[str], source_max_length=1224, candidate_max_length=412):
#     ids = []
#     assert len(sources) == len(candidate1s) == len(candidate2s)
#     max_length = source_max_length + 2 * candidate_max_length
#     for i in range(len(sources)):
#         source_ids = tokenizer.encode(source_prefix + sources[i], max_length=source_max_length, truncation=True)
#         candidate_max_length = (max_length - len(source_ids)) // 2
#         candidate1_ids = tokenizer.encode(cand1_prefix + candidate1s[i], max_length=candidate_max_length, truncation=True)
#         candidate2_ids = tokenizer.encode(cand2_prefix + candidate2s[i], max_length=candidate_max_length, truncation=True)
#         ids.append(source_ids + candidate1_ids + candidate2_ids)
#     encodings = tokenizer.pad({"input_ids": ids}, return_tensors="pt", padding="max_length", max_length=max_length)
#     return encodings

# encodings = tokenize_pair(inputs, candidates_A, candidates_B)
# encodings = {k:v.to(pairrm.device) for k,v in encodings.items()}
# outputs = pairrm(**encodings)
# logits = outputs.logits.tolist()
# comparison_results = outputs.logits > 0
# print(logits)
# # [1.9003021717071533, -1.2547134160995483]
# print(comparison_results)